In [1]:
import pandas as pd
import numpy as np
import re

from torch import nn
import nltk
import pandas as pd
import matplotlib.pyplot as plt
from sentence_transformers import *
import torchvision.models as models


import torch.nn as nn
from transformers import BertModel, BertTokenizer
import torch.nn as nn
import torch.nn.functional as F
from sentence_transformers import SentenceTransformer, util
from sklearn.model_selection import train_test_split
import seaborn as sns
from sklearn.neighbors import NearestNeighbors
from torch.nn import Transformer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score


# import pre-processed data
import pandas as pd
import numpy as np
import os
#import cv2
import matplotlib.pyplot as plt
import PIL
import tensorflow as tf
#import keras_toolkit as kt
import keras.backend as K
from sklearn.model_selection import train_test_split
from textwrap import wrap

/usr/local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# process text
import string
import re
from nltk.corpus import stopwords
import pandas as pd

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# remove numbers
def remove_numbers(text):
    'Remove all the numbers from the text'
    result = re.sub(r'\d+', '', text)
    return result

# remove special characters
def remove_special_characters(text):
    'Remove all the special characters from the text'
    pattern = r'[^a-zA-z0-9\s]'
    text = re.sub(pattern, '', text)
    return text

# remove extra spaces
def remove_extra_spaces(text):
    'remove extra spaces from the text'
    text = re.sub(' +', ' ', text)
    return text

def word_tokenize(text):
    'tokenize the text'
    text = text.split()
    return text

# remove stop words
def remove_stop_words(text):
    'remove stop words from the text'
    text_tokens = word_tokenize(text)
    tokens_without_sw = [word for word in text_tokens if not word in stop_words]
    filtered_sentence = (" ").join(tokens_without_sw)
    return filtered_sentence

# remove all preprocessing
def remove_all_preprocessing(text):
    'Remove all the preprocessing from the text'
    text = remove_numbers(text)
    text = remove_special_characters(text)
    text = remove_extra_spaces(text)
    text = remove_stop_words(text)
    return text

def process_text(training_dataset, val_dataset, testing_dataset):
    'run all process above to clean titles'

    stop_words = set(stopwords.words('english'))
    punctuations = string.punctuation

    # apply all preprocessing
    training_dataset['proc_title'] = training_dataset['title'].apply(lambda x: remove_all_preprocessing(x))
    val_dataset['proc_title'] = val_dataset['title'].apply(lambda x: remove_all_preprocessing(x))
    testing_dataset['proc_title'] = testing_dataset['title'].apply(lambda x: remove_all_preprocessing(x))

    # get rid of \
    training_dataset['proc_title'] = training_dataset['proc_title'].apply(lambda x: x.replace('\\', ''))
    val_dataset['proc_title'] = val_dataset['proc_title'].apply(lambda x: x.replace('\\', ''))
    testing_dataset['proc_title'] = testing_dataset['proc_title'].apply(lambda x: x.replace('\\', ''))
    # lower case
    training_dataset['proc_title'] = training_dataset['proc_title'].apply(lambda x: x.lower())
    val_dataset['proc_title'] = val_dataset['proc_title'].apply(lambda x: x.lower())
    testing_dataset['proc_title'] = testing_dataset['proc_title'].apply(lambda x: x.lower())

    return training_dataset, val_dataset, testing_dataset


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
# detect and init the TPU
tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()

# instantiate a distribution strategy
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.
INFO:tensorflow:Initializing the TPU system: local
INFO:tensorflow:Finished initializing TPU system.


INFO:tensorflow:Found TPU system:


INFO:tensorflow:Found TPU system:


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Cores: 8


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:6, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


In [4]:

training_dataset = pd.read_csv('/kaggle/input/15-after-processed/15_after_processed/train.csv')
val_dataset = pd.read_csv('/kaggle/input/15-after-processed/15_after_processed/val.csv')
testing_dataset = pd.read_csv('/kaggle/input/15-after-processed/15_after_processed/test.csv')
training_dataset, val_dataset, testing_dataset = process_text(training_dataset, val_dataset, testing_dataset)
training_dataset.head()

,posting_id,image,image_phash,title,label_group,proc_title
0,train_2249428809,40d304f91be807fed3b796a158a034f7.jpg,bc69d33e86b491c1,AMTECH Klem C Set 3 Pcs - Catok Clamp 1 2 3 Inch,3685949317,amtech klem c set pcs catok clamp inch
1,train_1453612941,e05aa71652e2546dbc2a6a2d18cc4aa7.jpg,bcfcc1a80ee930da,Cetakan Kue Pukis 10 Lubang Pancong Teflon Ran...,2236232282,cetakan kue pukis lubang pancong teflon rangin...
2,train_414717665,0c0f97f726132be83c7b84e0ca9e604e.jpg,bc4aa5a5de34d2e0,(COD)TAS TOTE BAG T76 TAS CANVAS FASHION WANIT...,3145779110,codtas tote bag t tas canvas fashion wanita im...
3,train_404098311,fedf1394058d6e439698356c0be6f4d6.jpg,cbceb1a14e8bf046,(1kg=7pcs) MEIRA CARDIGAN CARDI OUTHER TANPA J...,4171236554,kgpcs meira cardigan cardi outher tanpa jilbab
4,train_653040440,cfb08184ebbef7a5409e1bd1a0f6d0d9.jpg,c44eec61e516d2b9,Wardah Lightening Day Cream 30gr or night cre...,1774190279,wardah lightening day cream gr night cream gr


In [5]:
with tpu_strategy.scope():
    dist_model = SentenceTransformer('stsb-distilbert-base')
    dist_model.max_seq_length = 128

train_titles = training_dataset.proc_title.tolist()
train_embed = dist_model.encode(train_titles, show_progress_bar=True, convert_to_tensor=True)

val_titles = val_dataset.proc_title.tolist()
val_embed = dist_model.encode(val_titles, show_progress_bar=True, convert_to_tensor=True)

Batches: 100%|██████████| 172/172 [00:11<00:00, 14.65it/s]


In [ ]:
IMG_SIZE = 224
size = (IMG_SIZE,IMG_SIZE)
with tpu_strategy.scope():
    img_model = tf.keras.applications.MobileNet(input_shape = (IMG_SIZE, IMG_SIZE, 3), include_top = False, weights = 'imagenet' )
    

In [ ]:
def get_textEmbeddings(model,text):
    text_embedding = model.encode(text, convert_to_tensor=True)
    return text_embedding

def get_imageEmbeddings(model,imagePath):
    image = tf.keras.preprocessing.image.load_img(imagePath,target_size= size)
    input_arr = tf.keras.preprocessing.image.img_to_array(image)
    input_arr = np.array([input_arr])
    img_embeddings = model(input_arr)
    meanImgEmb1 = np.mean(img_embeddings,axis =0)
    meanImgEmb2 = np.mean(meanImgEmb1,axis=0)
    meanImgEmb = np.mean(meanImgEmb2,axis=0)
    return meanImgEmb

X_train, X_val, y_train, y_val = train_embed, val_embed, training_dataset.label_group, val_dataset.label_group

In [ ]:
text_embeddings={}
image_embeddings={}
with tpu_strategy.scope():
    for index,row in  training_dataset.iterrows():
        txt_emb = get_textEmbeddings(dist_model,str(row['proc_title']))
        imagePath = '/kaggle/input/shopee-product-matching/train_images/'+row[1]
        img_emb = get_imageEmbeddings(img_model,imagePath)
        text_embeddings[row[0]] = txt_emb
        image_embeddings[row[0]] = img_emb


In [ ]:
val_text_embeddings={}
val_image_embeddings={}
with tpu_strategy.scope():
    for index,row in  val_dataset.iterrows():
        txt_emb = get_textEmbeddings(dist_model,str(row['proc_title']))
        imagePath = '/kaggle/input/shopee-product-matching/train_images/'+row[1]
        img_emb = get_imageEmbeddings(img_model,imagePath)
        val_text_embeddings[row[0]] = txt_emb
        val_image_embeddings[row[0]] = img_emb

test_text_embeddings={}
test_image_embeddings={}
with tpu_strategy.scope():
    for index,row in  testing_dataset.iterrows():
        txt_emb = get_textEmbeddings(dist_model,str(row['proc_title']))
        imagePath = '/kaggle/input/shopee-product-matching/train_images/'+row[1]
        img_emb = get_imageEmbeddings(img_model,imagePath)
        test_text_embeddings[row[0]] = txt_emb
        test_image_embeddings[row[0]] = img_emb

In [ ]:
keyList=[]
cembList=[]
imageList=[]
titleList=[]
for index, row in training_dataset.iterrows():
    txt_emb = text_embeddings[row[0]].cpu().numpy()
    img_emb = image_embeddings[row[0]]
    cmb_emb = np.concatenate((txt_emb,img_emb),axis=0)

    norm = np.linalg.norm(cmb_emb)
    cmb_emb_normal = cmb_emb/norm
    
    keyList.append(row[0])
    cembList.append(cmb_emb_normal)
    imageList.append(row['image'])
    titleList.append(row['proc_title'])

In [ ]:
acc_dict = {}
f1_dict = {}
X_train, X_val, y_train, y_val = cembList, val_cembList, training_dataset.label_group, val_dataset.label_group


from sklearn.metrics import accuracy_score, f1_score
for k in [1, 2, 3, 5, 10, 25, 50, 100]:
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train, y_train)
    y_pred = knn.predict(X_val)

    # Evaluate the performance on the test set
    accuracy = accuracy_score(y_val, y_pred)
    f1 = f1_score(y_val, y_pred, average='weighted')
    acc_dict[k] = accuracy
    f1_dict[k] = f1


In [ ]:
plt.plot(acc_dict.keys(), acc_dict.values(), label='accuracy')
plt.plot(f1_dict.keys(), f1_dict.values(), label='f1 score')
plt.legend()
plt.title('Tuning Number of Neighbors')
plt.xlabel('Number of Neighbors')
plt.ylabel('Score')

In [ ]:
keyList=[]
test_cembList=[]
test_imageList=[]
test_titleList=[]

for index, row in testing_dataset.iterrows():
    txt_emb = test_text_embeddings[row[0]].cpu().numpy()
    img_emb = test_image_embeddings[row[0]]
    cmb_emb = np.concatenate((txt_emb,img_emb),axis=0)

    norm = np.linalg.norm(cmb_emb)
    cmb_emb_normal = cmb_emb/norm
    
    keyList.append(row[0])
    test_cembList.append(cmb_emb_normal)
    test_imageList.append(row['image'])
    test_titleList.append(row['proc_title'])

In [ ]:
X_test, y_test = test_cembList, testing_dataset.label_group

knn = KNeighborsClassifier(n_neighbors=2)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')
print(f'Accuracy = {accuracy}')
print(f'F1 score = {f1}')